In [8]:
%load_ext autoreload
%autoreload 2
import TRITON_SWMM_toolkit.examples as examples
import TRITON_SWMM_toolkit.analysis as analysis
from TRITON_SWMM_toolkit.process_simulation import TRITONSWMM_sim_post_processing
import TRITON_SWMM_toolkit.process_simulation as ps
import TRITON_SWMM_toolkit.processing_analysis as pe
from TRITON_SWMM_toolkit.process_simulation import *
import TRITON_SWMM_toolkit.plot_utils as plot_utils
import TRITON_SWMM_toolkit.utils as utils 
import TRITON_SWMM_toolkit.run_simulation as run
import TRITON_SWMM_toolkit.sensitivity_analysis as snst
from TRITON_SWMM_toolkit.config import analysis_config
from TRITON_SWMM_toolkit.examples import GetTS_TestCases as tst
from importlib import reload
reload(examples)
reload(analysis)
reload(ps)
reload(pe)
reload(plot_utils)
reload(utils)
reload(snst)
# norfolk_singlesim = examples.GetTS_TestCases.retreive_norfolk_single_sim_test_case(download_if_exists=False)
# norfolk_multisim = examples.GetTS_TestCases.retreive_norfolk_multi_sim_test_case()

rebuild_case = True
nrflk_multisim_ensemble = tst.retreive_norfolk_multi_sim_test_case(start_from_scratch=rebuild_case)
self = nrflk_multisim_ensemble.system.analysis
nrflk_multisim_ensemble.system.process_system_level_inputs(overwrite_if_exists=rebuild_case)
self.compile_TRITON_SWMM(recompile_if_already_done_successfully=rebuild_case)
# self.sensitivity.prepare_scenarios_in_each_subanalysis()
# self.prepare_all_scenarios()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
swig/python detected a memory leak of type 'struct Handle *', no destructor found.


In [ ]:
launcher_logfile = self.scenarios[0].log.logfile.parent / "scenario_prep.log"
prepare_scenario_launchers = self.retrieve_prepare_scenario_launchers(
    overwrite_scenario=True, verbose=True
)
log_path = prepare_scenario_launchers[0]()
print(f"Log file written to {log_path}")


... SWMM Version 5.2.4
 o  Retrieving project data
 o  Simulating day: 0     hour: Simulation complete           
... Run Complete

PosixPath('/home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/sims/0-event_id.0/scenario_prep.log')

In [ ]:

self.run_python_functions_concurrently(prepare_scenario_launchers)
scens_not_created = "\n".join(self.scenarios_not_created)
print(scens_not_created)
# self.scenarios_not_created

Memory-based limit: 2 (available 2441 MiB, 1024 MiB per task)
Running 2 functions (max parallel = 1)

... SWMM Version 5.2.4
... Run Complete
 o  Retrieving project data
 o  Simulating day: 0     hour: Simulation complete           Function 0 completed in 3.61 s

... SWMM Version 5.2.4
 o  Retrieving project data
 o  Simulating day: 0     hour: Simulation complete           
... Run CompleteFunction 1 completed in 2.86 s



Exception: ERROR 440: an unspecified error has occurred

# debugging sensitivity analysis stuff

In [3]:
nrflk_cpu_sensitivity = tst.retreive_norfolk_cpu_config_sensitivity_case(
    start_from_scratch=True
)
analysis = nrflk_cpu_sensitivity.system.analysis
analysis.compile_TRITON_SWMM(recompile_if_already_done_successfully=True)
assert analysis.compilation_successful == True
# prepare scenarios
analysis.sensitivity.prepare_scenarios_in_each_subanalysis(concurrent=True)
assert analysis.log.all_scenarios_created.get() == True
# run sims
analysis.sensitivity.run_all_sims(pickup_where_leftoff=False, concurrent=True)
assert analysis.log.all_sims_run.get() == True


... SWMM Version 5.2.4
... Run Complete
 o  Retrieving project data
 o  Simulating day: 0     hour: Simulation complete           
... SWMM Version 5.2.4
 o  Retrieving project data
... Run Complete
 o  Simulating day: 0     hour: Simulation complete           
... SWMM Version 5.2.4
 o  Retrieving project data
... Run Complete
 o  Simulating day: 0     hour: Simulation complete           
... SWMM Version 5.2.4
 o  Retrieving project data
... Run Complete
 o  Simulating day: 0     hour: Simulation complete           

In [ ]:
# consolidate outputs
analysis.sensitivity.process_simulation_timeseries_concurrently(
        overwrite_if_exist=True
    )

In [3]:
# running multiple sims concurrently
launch_functions = self._create_launchable_sims()
self.run_simulations_concurrently_on_desktop(launch_functions=launch_functions, verbose = True)

Running up to 4 simulations concurrently
Simulation finished: simulation completed
Simulation finished: simulation completed


['simulation completed', 'simulation completed']

In [ ]:
self.run_simulations_concurrently_on_SLURM_HPC(launch_functions)

In [2]:
# running single sim 
self.run_sim(0, pickup_where_leftoff=False, process_outputs_after_sim_completion = False, which = "both", clear_raw_outputs=False, overwrite_if_exist=False, compression_level=5, verbose = True)


run instance instantiated
running TRITON-SWMM simulatoin for event 0-event_id.0
bash command to view progress:
tail -f /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/sims/0-event_id.0/tritonswmm_sim_logfiles/2026-01-14T105454-0500.log


running TRITON-SWMM simulatoin for event 0-event_id.0
bash command to view progress:
tail -f /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/sims/0-event_id.0/tritonswmm_sim_logfiles/2026-01-14T104700-0500.log


In [7]:
elapsed

8.297805070877075

In [ ]:
from TRITON_SWMM_toolkit import examples

examples.load_norfolk_system_config(download_if_exists=True)

signed into Hydroshare successfully.
Retrieving /hsapi/resource/a4aace329b8c401a93e94ce2a761fe1b


PosixPath('/home/dcl3nd/.local/share/TRITON_SWMM_toolkit/examples/norfolk_coastal_flooding/data/contents/system/config_system.yaml')

In [2]:
run_command = self.retrieve_sim_command_text(in_slurm = True)
self.create_ensemble_bash_script(run_command)

print(utils.read_text_file_as_string(self.analysis_paths.bash_script_path))

#!/bin/bash
#SBATCH -o _slurm_outputs/%x/%A_outputs_%a_%N.out
#SBATCH -e _slurm_outputs/%x/%A_errors_%a_%N.out
#SBATCH -A CLI190
#SBATCH -t 02:00:00
#SBATCH -p batch
#SBATCH -N 1
#SBATCH --exclusive
# SBATCH --cpus-per-task=${cpus_per_task}   # CPU=1, GPU=threads per GPU
# SBATCH --ntasks-per-node=${ntasks_per_node} # GPU=tasks per node, CPU optional
# SBATCH --gres=None
#SBATCH --mail-user=dcl3nd@virginia.edu
#SBATCH --mail-type=ALL
#SBATCH -q debug

set -euo pipefail

# Load modules
module purge
module load PrgEnv-amd Core/24.07 cmake/3.27.9 craype-accel-amd-gfx90a
module load miniforge3/23.11.0
DIR=~/.conda/envs/running_swmm

echo "Node CPUs: $(nproc)"
echo "SLURM_CPUS_ON_NODE=${SLURM_CPUS_ON_NODE:-unset}"

export ROCM_PATH=${CRAY_AMD_COMPILER_PREFIX}
export CXX=CC
export MPICH_GPU_SUPPORT_ENABLED=1
export TRITON_BACKEND="Kokkos_ENABLE_HIP"
export TRITON_ARCH="Kokkos_ARCH_AMD_GFX90A"
export TRITON_CXX_FLAGS="-DTRITON_HIP_LAUNCHER;-O3;-ffast-math;-I${ROCM_PATH}/include;-D__HIP_ROCclr

In [ ]:
launch_functions = self._create_launchable_sims(pickup_where_leftoff=False, verbose = True)
self.run_simulations_concurrently_on_desktop(launch_functions, use_gpu = False, total_gpus_available=0, verbose=True)

running TRITON-SWMM simulatoin for event 0-event_id.0
bash command to view progress:
tail -f /home/dcl3nd/.local/share/TRITON_SWMM_toolkit/examples/norfolk_coastal_flooding/data/contents/tests/cpu_config_sensitivity/sims/0-event_id.0/tritonswmm_sim_logfiles/2026-01-13T170838-0500.log
running TRITON-SWMM simulatoin for event 1-event_id.1
bash command to view progress:
tail -f /home/dcl3nd/.local/share/TRITON_SWMM_toolkit/examples/norfolk_coastal_flooding/data/contents/tests/cpu_config_sensitivity/sims/1-event_id.1/tritonswmm_sim_logfiles/2026-01-13T170838-0500.log
Running up to 4 sims concurrently
Simulation 1 finished: simulation completed
Simulation 0 finished: simulation completed


['simulation completed', 'simulation completed']

In [6]:
self.process_all_sim_timeseries(which = "both", clear_raw_outputs=True, overwrite_if_exist=True, verbose = True)

Time to load bin triton outputs (min) 0.04
finished writing /home/dcl3nd/.local/share/TRITON_SWMM_toolkit/examples/norfolk_coastal_flooding/data/contents/tests/cpu_config_sensitivity/sims/0-event_id.0/TRITON_tseries.zarr
##################################
Found problem. Orifice conduits do not return max velocity or max over full flow. Filling with empty string
Normal row vs. problem row:
  0                    CONDUIT     0.000     0  00:01      0.00    0.00    0.01

  756                  ORIFICE     0.000     0  00:00                      0.00

Properly parsed values:
['756', 'ORIFICE', '0.000', '0', '00:00', '', '', '0.00\n']
Converted variable to datatype = type, <class 'str'>
Converted variable to datatype = OutfallType, <class 'str'>
Converted variable to datatype = StageOrTimeseries, <class 'str'>
Converted variable to datatype = StorageCurve, <class 'str'>
Converted variable to datatype = Coefficient, <class 'str'>
Converted variable to datatype = type, <class 'str'>
Converted

/home/dcl3nd/dev/TRITON-SWMM_toolkit/src/TRITON_SWMM_toolkit/process_simulation.py:493: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'node_id' ('node_id',) The recommendation is to set join explicitly for this case.
  ds_nodes = xr.merge(
/home/dcl3nd/dev/TRITON-SWMM_toolkit/src/TRITON_SWMM_toolkit/process_simulation.py:500: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'link_id' ('link_id',) The recommendation is to set join explicitly for this case.
  ds_links = xr.merge(
/home/dcl3nd/miniconda3/envs/triton_sw

finished writing /home/dcl3nd/.local/share/TRITON_SWMM_toolkit/examples/norfolk_coastal_flooding/data/contents/tests/cpu_config_sensitivity/sims/0-event_id.0/SWMM_node_tseries.zarr


/home/dcl3nd/miniconda3/envs/triton_swmm_toolkit/lib/python3.11/site-packages/zarr/core/dtype/npy/string.py:249: UnstableSpecificationWarning: The data type (FixedLengthUTF32(length=13, endianness='little')) does not have a Zarr V3 specification. That means that the representation of arrays saved with this data type may change without warning in a future version of Zarr Python. Arrays stored with this data type may be unreadable by other Zarr libraries. Use this data type at your own risk! Check https://github.com/zarr-developers/zarr-extensions/tree/main/data-types for the status of data type specifications for Zarr V3.
  v3_unstable_dtype_warning(self)
/home/dcl3nd/miniconda3/envs/triton_swmm_toolkit/lib/python3.11/site-packages/zarr/core/dtype/npy/string.py:249: UnstableSpecificationWarning: The data type (FixedLengthUTF32(length=10, endianness='little')) does not have a Zarr V3 specification. That means that the representation of arrays saved with this data type may change without 

finished writing /home/dcl3nd/.local/share/TRITON_SWMM_toolkit/examples/norfolk_coastal_flooding/data/contents/tests/cpu_config_sensitivity/sims/0-event_id.0/SWMM_link_tseries.zarr
Time to load bin triton outputs (min) 0.04
finished writing /home/dcl3nd/.local/share/TRITON_SWMM_toolkit/examples/norfolk_coastal_flooding/data/contents/tests/cpu_config_sensitivity/sims/1-event_id.1/TRITON_tseries.zarr
##################################
Found problem. Orifice conduits do not return max velocity or max over full flow. Filling with empty string
Normal row vs. problem row:
  0                    CONDUIT     0.000     0  00:01      0.00    0.00    0.01

  756                  ORIFICE     0.000     0  00:00                      0.00

Properly parsed values:
['756', 'ORIFICE', '0.000', '0', '00:00', '', '', '0.00\n']
Converted variable to datatype = type, <class 'str'>
Converted variable to datatype = OutfallType, <class 'str'>
Converted variable to datatype = StageOrTimeseries, <class 'str'>
Co

/home/dcl3nd/dev/TRITON-SWMM_toolkit/src/TRITON_SWMM_toolkit/process_simulation.py:493: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'node_id' ('node_id',) The recommendation is to set join explicitly for this case.
  ds_nodes = xr.merge(
/home/dcl3nd/dev/TRITON-SWMM_toolkit/src/TRITON_SWMM_toolkit/process_simulation.py:500: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'link_id' ('link_id',) The recommendation is to set join explicitly for this case.
  ds_links = xr.merge(
/home/dcl3nd/miniconda3/envs/triton_sw

finished writing /home/dcl3nd/.local/share/TRITON_SWMM_toolkit/examples/norfolk_coastal_flooding/data/contents/tests/cpu_config_sensitivity/sims/1-event_id.1/SWMM_node_tseries.zarr


/home/dcl3nd/miniconda3/envs/triton_swmm_toolkit/lib/python3.11/site-packages/zarr/core/dtype/npy/string.py:249: UnstableSpecificationWarning: The data type (FixedLengthUTF32(length=13, endianness='little')) does not have a Zarr V3 specification. That means that the representation of arrays saved with this data type may change without warning in a future version of Zarr Python. Arrays stored with this data type may be unreadable by other Zarr libraries. Use this data type at your own risk! Check https://github.com/zarr-developers/zarr-extensions/tree/main/data-types for the status of data type specifications for Zarr V3.
  v3_unstable_dtype_warning(self)
/home/dcl3nd/miniconda3/envs/triton_swmm_toolkit/lib/python3.11/site-packages/zarr/core/dtype/npy/string.py:249: UnstableSpecificationWarning: The data type (FixedLengthUTF32(length=10, endianness='little')) does not have a Zarr V3 specification. That means that the representation of arrays saved with this data type may change without 

finished writing /home/dcl3nd/.local/share/TRITON_SWMM_toolkit/examples/norfolk_coastal_flooding/data/contents/tests/cpu_config_sensitivity/sims/1-event_id.1/SWMM_link_tseries.zarr


In [9]:
self.log.all_sims_run.get()

True